# kmz to csv converter for SITG geodata

Geodata from the [SITG catalog](http://ge.ch/sitg/sitg_catalog/sitg_donnees) contain either good data but a very unconvenient geodata format, or good geodata and all other data nested in a very unconvenient `<description>` tag. This script extracts and merges:
* data from the **csv archive** (example: CSV_GOL_DECHETTERIE.zip)
* data from the **kmz archive** (example: KML_GOL_DECHETTERIE)

You need to download both file formats of the dataset. No need to extract the contents of the zip, the script does it for you.

In [1]:
from zipfile import ZipFile
import glob

# Unzip


In [2]:
zipFiles = glob.glob('*.zip')
zipFiles

['CSV_OTC_AMENAG_2ROUES.zip', 'KML_OTC_AMENAG_2ROUES.zip']

In [15]:
kmz_filename = ''
csv_filename = ''

for zipFile in zipFiles:
    folderContent = ZipFile(zipFile, 'r')
    filenames = [item.filename for item in folderContent.filelist]
    print('Extracting', filenames[0])
    
    if filenames[0][-4:] == '.kmz':
        kmz_filename = filenames[0]
        print('>> kmz file found:', kmz_filename)
    elif filenames[0][-4:] == '.csv':
        csv_filename = filenames[0]
        print('>> csv file found:', csv_filename)
    
    folderContent.extract(filenames[0])
    folderContent.close()

Extracting OTC_AMENAG_2ROUES.csv
>> csv file found: OTC_AMENAG_2ROUES.csv
Extracting OTC_AMENAG_2ROUES.kmz
>> kmz file found: OTC_AMENAG_2ROUES.kmz


# Extract kmz

In [16]:
kmz = ZipFile(kmz_filename, 'r')
kml = kmz.open('doc.kml', 'r')

In [17]:
import xml.sax, xml.sax.handler
class PlacemarkHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.inName = False # handle XML parser events
        self.inPlacemark = False
        self.mapping = {}
        self.buffer = ""
        self.name_tag = ""
       
    def startElement(self, name, attributes):
        if name == "Placemark": # on start Placemark tag
            self.inPlacemark = True
            self.buffer = ""
        if self.inPlacemark:
            if name == "name": # on start title tag
                self.inName = True # save name text to follow
           
    def characters(self, data):
        if self.inPlacemark: # on text within tag
            self.buffer += data # save text if in title
           
    def endElement(self, name):
        self.buffer = self.buffer.strip('\n\t')
       
        if name == "Placemark":
            self.inPlacemark = False
            self.name_tag = "" #clear current name
       
        elif name == "name" and self.inPlacemark:
            self.inName = False # on end title tag           
            self.name_tag = self.buffer.strip()
            self.mapping[self.name_tag] = {}
        elif self.inPlacemark:
            if name in self.mapping[self.name_tag]:
                self.mapping[self.name_tag][name] += self.buffer
            else:
                self.mapping[self.name_tag][name] = self.buffer
        self.buffer = ""

In [18]:
parser = xml.sax.make_parser()
handler = PlacemarkHandler()
parser.setContentHandler(handler)
parser.parse(kml)
kmz.close()

In [34]:
def build_table(mapping):
    sep = ';'
    count = 0
    
    output = 'Name' + sep + 'Coordinates\n'
    points = ''
    lines = ''
    shapes = ''
    for key in mapping:
        coord_str = mapping[key]['coordinates'] + sep
       
        if 'LookAt' in mapping[key]: #points
            points += key + sep + coord_str + "\n"
        elif 'LineString' in mapping[key]: #lines
            lines += key + sep + coord_str + "\n"
        else: #shapes
            shapes += key + sep + coord_str + "\n"
        count += 1
    output += points + lines + shapes
    print(count, "objects found.")
    return output

In [59]:
printhandler

{'': {'LineString': '        ',
  'MultiGeometry': '      ',
  'Snippet': '      ',
  'altitudeMode': 'clampToGround',
  'coordinates': '           6.128479810665287,46.24692898799965,0 6.12856218788056,46.24698149590602,0 6.128695220118461,46.24707372783185,0 6.128833782123142,46.2472217143953,0 6.128935311070237,46.24738357626659,0 6.129009548954826,46.24753711429248,0',
  'description': '      <html xmlns:fo="http://www.w3.org/1999/XSL/Format" xmlns:msxsl="urn:schemas-microsoft-com:xslt">\n\n<head>\n\n<META http-equiv="Content-Type" content="text/html">\n\n<meta http-equiv="content-type" content="text/html; charset=UTF-8">\n\n</head>\n\n<body style="margin:0px 0px 0px 0px;overflow:auto;background:#FFFFFF;">\n\n<table style="font-family:Arial,Verdana,Times;font-size:12px;text-align:left;width:100%;border-collapse:collapse;padding:3px 3px 3px 3px">\n\n<tr style="text-align:center;font-weight:bold;background:#9CBCE2">\n\n<td> </td>\n\n</tr>\n\n<tr>\n\n<td>\n\n<table style="font-family:

In [35]:
outstr = build_table(handler.mapping)
out_filename = filename[:-4] + "-geo.csv" #output filename same as input plus .csv
f = open(out_filename, "w")
f.write(outstr)
f.close()
print(outstr)

624 objects found.
Name;Coordinates
;           6.128479810665287,46.24692898799965,0 6.12856218788056,46.24698149590602,0 6.128695220118461,46.24707372783185,0 6.128833782123142,46.2472217143953,0 6.128935311070237,46.24738357626659,0 6.129009548954826,46.24753711429248,0;
59587;           6.13026533560365,46.21512138872929,0 6.130048277952455,46.21528720315327,0 6.130012794845467,46.21534527127337,0 6.129822490250382,46.21549259208185,0 6.129555401372452,46.21569075383069,0 6.129212798218488,46.21591385976808,0 6.129162966831852,46.21593262479657,0 6.128872591145666,46.21611270098107,0 6.128512194633752,46.21633712781122,0 6.128411389736358,46.21643276372539,0 6.1281502819504,46.21660283153063,0 6.127964019140699,46.21672725425687,0 6.127794256441016,46.21682847649938,0 6.127742622287975,46.21687358079318,0;
59455;           6.141969471275596,46.19520856977934,0 6.141969094555117,46.19517545711968,0;
07129;           6.142375131060816,46.20766916419498,0 6.142269185069573,46.20744941

# Merge

In [21]:
import pandas as pd

In [26]:
df = pd.read_csv(csv_filename, delimiter=';', encoding='Windows 1252')
geo = pd.read_csv(csv_filename[:-4] + '-geo.csv', delimiter=';')
print("df length =", len(df), "\ngeodata length =", len(geo))

df length = 4156 
geodata length = 624


In [54]:
geo['id'] = geo.index.astype(int)

In [55]:
geo

,Name,Coordinates,id
NaN,"6.128479810665287,46.24692898799965...",NaN,-2147483648
59587.0,"6.13026533560365,46.21512138872929,...",NaN,59587
59455.0,"6.141969471275596,46.19520856977934...",NaN,59455
7129.0,"6.142375131060816,46.20766916419498...",NaN,7129
5266.0,"6.150087802464072,46.21525383920695...",NaN,5266
10022.0,"6.144362617483273,46.19073800166993...",NaN,10022
5932.0,"6.203333396113067,46.22133089097074...",NaN,5932
17779.0,"6.153968511904792,46.19367146369973...",NaN,17779
5185.0,"6.101927178956148,46.20657814817916...",NaN,5185
2534.0,"6.099231184021472,46.2089042683445,...",NaN,2534


In [30]:
geo.head()

,Name,Coordinates,id
NaN,"6.128479810665287,46.24692898799965...",NaN,NaN
59587.0,"6.13026533560365,46.21512138872929,...",NaN,59587.0
59455.0,"6.141969471275596,46.19520856977934...",NaN,59455.0
7129.0,"6.142375131060816,46.20766916419498...",NaN,7129.0
5266.0,"6.150087802464072,46.21525383920695...",NaN,5266.0


In [29]:
df.head()

,OBJECTID,CODE_VOIE,NOM_VOIE,TYPE_AMENAGEMENT,REALISATION,CIRCUL2R,CIRCVOIT,AFFECTATION,TOURNGAUCHE,SHAPE.LEN
0,57170,15482,Route de Meyrin,Bande,2009,unidirectionnelle,dans le sens,vélos + cyclomoteurs,NaN,19.060129
1,57171,15482,Route de Meyrin,Bande,2009,unidirectionnelle,dans le sens,vélos + cyclomoteurs,NaN,41.714657
2,57172,15482,Route de Meyrin,Bande,2009,unidirectionnelle,dans le sens,vélos + cyclomoteurs,NaN,47.970454
3,57173,15482,Route de Meyrin,Bande,2011,unidirectionnelle,dans le sens,vélos + cyclomoteurs,NaN,17.087873
4,56543,11932,Avenue des Grandes-Communes,Bande,,unidirectionnelle,dans le sens,vélos + cyclomoteurs,NaN,28.843803


In [56]:
#df2 = df.merge(geo, left_on='NUMERO_SITE', right_on='id')
df2 = df.merge(geo, left_on='OBJECTID', right_on='id')

In [57]:
df2.shape
df2.head()
df2.to_csv(csv_filename[:-4]+'_export.csv', encoding='utf-8', delimiter=';', index=False)

In [36]:
df['TYPE_AMENAGEMENT'].value_counts()

Bande            2232
Piste            1132
Sas               216
Contresens        203
Dérogation 2R     202
Voie Bus          169
Name: TYPE_AMENAGEMENT, dtype: int64

In [38]:
df[df['TYPE_AMENAGEMENT'] == 'Dérogation 2R']

,OBJECTID,CODE_VOIE,NOM_VOIE,TYPE_AMENAGEMENT,REALISATION,CIRCUL2R,CIRCVOIT,AFFECTATION,TOURNGAUCHE,SHAPE.LEN
29,58312,25380,la Vy-des-Champs,Dérogation 2R,1986,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,191.387021
31,58311,25380,la Vy-des-Champs,Dérogation 2R,1986,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,456.316647
67,57334,16829,Place de la Navigation,Dérogation 2R,2000,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,1.858279
68,57335,16829,Place de la Navigation,Dérogation 2R,2000,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,56.626833
69,57336,16829,Place de la Navigation,Dérogation 2R,2000,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,28.086509
85,57356,16934,Rue Neuve-du-Molard,Dérogation 2R,2001,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,1.562370
86,57357,16934,Rue Neuve-du-Molard,Dérogation 2R,2001,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,74.669083
111,57390,17159,Avenue du Pailly,Dérogation 2R,2009,unidirectionnelle,,mixité piétons,NaN,17.687503
127,57408,17450,Rue PATRU,Dérogation 2R,1993,bidirectionnelle,,au statut de la rue : vélos seuls,NaN,167.049835
153,57146,15237,Rue MAURICE,Dérogation 2R,1996,bidirectionnelle,,au statut de la rue,NaN,31.080286


In [44]:
df[df['TYPE_AMENAGEMENT'] == 'Voie Bus']

,OBJECTID,CODE_VOIE,NOM_VOIE,TYPE_AMENAGEMENT,REALISATION,CIRCUL2R,CIRCVOIT,AFFECTATION,TOURNGAUCHE,SHAPE.LEN
3823,20487,13676,Rue LECT,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,47.741206
3824,12163,07129,Rue de Cornavin,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,81.377497
3825,20485,13676,Rue LECT,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,95.112934
3826,20486,13676,Rue LECT,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,57.029642
3827,24326,14346,Avenue du Mail,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,12.005253
3828,24002,NaN,,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,85.926761
3829,15685,03212,Rue BOVY-LYSBERG,Voie Bus,NaN,unidirectionnelle,contresens,vélos + cyclomoteurs + bus,NaN,60.104311
3830,16964,24210,Avenue TRONCHET,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,76.759058
3831,17282,22314,Rue de la Servette,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,24.688206
3832,17287,22993,Rue du Stand,Voie Bus,NaN,unidirectionnelle,dans le sens,vélos + cyclomoteurs + bus,NaN,19.459275


In [43]:
from math import nan
df[df['AFFECTATION'] != nan]['AFFECTATION'].value_counts()

vélos + cyclomoteurs                 3560
mixité piétons                        196
vélos + cyclomoteurs + bus            169
au statut de la rue                    77
au statut de la rue : vélos seuls      61
voie de présélection                    3
vélos seuls                             3
Name: AFFECTATION, dtype: int64

In [46]:
df2['AFFECTATION'].value_counts()

vélos + cyclomoteurs          13
vélos + cyclomoteurs + bus     4
Name: AFFECTATION, dtype: int64

In [51]:
geo

,Name,Coordinates,id
NaN,"6.128479810665287,46.24692898799965...",NaN,NaN
59587.0,"6.13026533560365,46.21512138872929,...",NaN,59587.0
59455.0,"6.141969471275596,46.19520856977934...",NaN,59455.0
7129.0,"6.142375131060816,46.20766916419498...",NaN,7129.0
5266.0,"6.150087802464072,46.21525383920695...",NaN,5266.0
10022.0,"6.144362617483273,46.19073800166993...",NaN,10022.0
5932.0,"6.203333396113067,46.22133089097074...",NaN,5932.0
17779.0,"6.153968511904792,46.19367146369973...",NaN,17779.0
5185.0,"6.101927178956148,46.20657814817916...",NaN,5185.0
2534.0,"6.099231184021472,46.2089042683445,...",NaN,2534.0


In [53]:
df['OBJECTID']

0       57170
1       57171
2       57172
3       57173
4       56543
5       66588
6       57181
7       58184
8       57175
9       62434
10      58051
11      56194
12      58583
13      58584
14      58585
15      58586
16      58587
17      58588
18      58589
19      58590
20      58592
21      58593
22      58595
23      58596
24      58597
25      58598
26      58599
27      58600
28      58601
29      58312
        ...  
4126      790
4127      803
4128      836
4129      840
4130      845
4131      928
4132      975
4133     1008
4134     1034
4135    24325
4136     7361
4137    15687
4138     1601
4139     4819
4140    21125
4141      224
4142    16324
4143      585
4144      274
4145      581
4146    21770
4147     1005
4148     1006
4149    17288
4150    21126
4151      807
4152      816
4153      133
4154      753
4155    25922
Name: OBJECTID, dtype: int64